In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import spacy
import nltk

In [5]:
df = pd.read_csv('data/articles.csv')
df.head()

,author,claps,reading_time,link,title,text
0,Justin Lee,8.3K,11,https://medium.com/swlh/chatbots-were-the-next...,Chatbots were the next big thing: what happene...,"Oh, how the headlines blared:\nChatbots were T..."
1,Conor Dewey,1.4K,7,https://towardsdatascience.com/python-for-data...,Python for Data Science: 8 Concepts You May Ha...,If you’ve ever found yourself looking up the s...
2,William Koehrsen,2.8K,11,https://towardsdatascience.com/automated-featu...,Automated Feature Engineering in Python – Towa...,Machine learning is increasingly moving from h...
3,Gant Laborde,1.3K,7,https://medium.freecodecamp.org/machine-learni...,Machine Learning: how to go from Zero to Hero ...,If your understanding of A.I. and Machine Lear...
4,Emmanuel Ameisen,935,11,https://blog.insightdatascience.com/reinforcem...,Reinforcement Learning from scratch – Insight ...,Want to learn about applied Artificial Intelli...


In [23]:
from spacy.lang.en import English

nlp = English()

doc = nlp(df.iloc[0].text)

In [44]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')

words = cv.fit_transform(df.text)
print(cv.get_feature_names())
print(words.toarray().sum())

['00', '000', '0000', '00000', '0001', '0002which', '00061', '00078', '000assuming', '000s', '000th', '000x', '001', '004', '0040424330518', '00634', '0078876', '01', '010', '011', '0157', '01783941', '019', '01x', '02', '0246238639582', '02627v1', '03', '034', '03640v1', '04', '048', '063', '072733877455676', '08', '083', '088', '09', '092', '09665', '0blue', '0jwnywjz6xhnrvayeclqpd', '0s', '0th', '0x3b', '10', '100', '1000', '10000', '1000s', '1000x', '1009', '100k', '100m', '100ms', '100s', '100th', '100x', '101', '1010bits', '1013', '1014synapses', '10170', '1024', '1024x1024', '1050', '106', '1060', '107', '1070', '1080', '1080atoms', '1080p', '1080s', '109additional', '109bits', '10am', '10e0', '10e3', '10fps', '10k', '10m', '10th', '10x', '11', '110', '11019', '1103', '110m', '111', '112', '11217', '11357', '1137', '114', '1143', '1159', '11786', '1188', '119', '12', '120', '1200', '1206', '120s', '123', '1231', '1231231', '1234567', '124', '125', '126', '1272942', '128', '1283'

1. Import the dataset
2. Clean it, keeping only English articles and words
3. Create a pipeline to remove stop words, count, lemmatize
4. Apply some NER from either spaCy or nltk to get topics
5. Create a pipeline to fit a corpus of  articles
    - Clean the data, removing stop words
    - Applying train_test_split
    - Fitting the model on training data
    - predicting on test data
    - outputting a MSE accuracy for regression
    
    
6. Pipeline for predicting on a specific article
7. Title predictor tool